In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-cqee_vlx
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-cqee_vlx
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=a634fcb83bc74d1131397b5cd26c012b39acd6b99f81d2481c22d1e3402ab14c
  Stored in directory: /tmp/pip-ephem-wheel-cache-v8qsezqx/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r /content/drive/'My Drive'/'Colab Notebooks'/buddy/requirements.txt

     |████████████████████████████████| 769 kB 13.7 MB/s 
     |████████████████████████████████| 3.0 MB 46.5 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 1.2 MB 50.0 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 


In [ ]:
import sys
sys.path.append('drive/My Drive/Colab Notebooks/')
sys.path.append('drive/My Drive/Colab Notebooks/buddy')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from tqdm import tqdm

import torch
from transformers import AdamW
from torch.utils.data import dataloader
from buddy.dataloader.wellness import WellnessTextClassificationDataset
from buddy.model.kobert import KoBERTforSequenceClassfication

평가함수


In [ ]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from tqdm import tqdm

import torch
from transformers import (
  AdamW,
  ElectraConfig,
  ElectraTokenizer
)
from torch.utils.data import dataloader
from dataloader.wellness import WellnessTextClassificationDataset
#from model.koelectra import koElectraForSequenceClassification
from model.kobert import KoBERTforSequenceClassfication
from kobert_transformers import get_tokenizer

logger = logging.getLogger(__name__)


MODEL_CLASSES ={
  #"koelectra": (ElectraConfig, koElectraForSequenceClassification, ElectraTokenizer),
  "kobert": (KoBERTforSequenceClassfication)
}
CHECK_POINT ={
  #"koelectra": "/content/drive/MyDrive/Colab Notebooks/buddy/checkpoint/koelectra-wellnesee-text-classification.pth",
  "kobert": "/content/drive/MyDrive/Colab Notebooks/buddy/checkpoint/kobert-wellness-text-classification-dataX-191.pth"
}

def get_model_and_tokenizer(model_name, device):
  save_ckpt_path = CHECK_POINT[model_name]

#  if model_name== "koelectra":
#    model_name_or_path = "monologg/koelectra-base-discriminator"

#    tokenizer = ElectraTokenizer.from_pretrained(model_name_or_path)
#    electra_config = ElectraConfig.from_pretrained(model_name_or_path)
#    model = koElectraForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path,
#                                                               config=electra_config,
#                                                               num_labels=359)
  if model_name =='kobert':
    tokenizer = get_tokenizer()
    model = KoBERTforSequenceClassfication()

  if os.path.isfile(save_ckpt_path):
      checkpoint = torch.load(save_ckpt_path, map_location=device)
      pre_epoch = checkpoint['epoch']
      # pre_loss = checkpoint['loss']
      model.load_state_dict(checkpoint['model_state_dict'])

      print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")

  return model, tokenizer

def get_model_input(data):
  if model_name =='kobert':
    return data
#  elif model_name== "koelectra":
#    return {'input_ids': data['input_ids'],
#              'attention_mask': data['attention_mask'],
#              'labels': data['labels']
#              }

def evaluate(model_name, device, batch_size, data_path):

  model, tokenizer = get_model_and_tokenizer(model_name, device)
  model.to(device)

  # WellnessTextClassificationDataset 데이터 로더
  eval_dataset = WellnessTextClassificationDataset(file_path=data_path,device=device, tokenizer=tokenizer)
  eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=True)

  logger.info("***** Running evaluation on %s dataset *****")
  logger.info("  Num examples = %d", len(eval_dataset))
  logger.info("  Batch size = %d", batch_size)

  loss = 0
  acc = 0


  # model.eval()
  for data in tqdm(eval_dataloader, desc="Evaluating"):
    with torch.no_grad():
      inputs = get_model_input(data)
      outputs = model(**inputs)
      loss += outputs[0]
      logit = outputs[1]
      acc += (logit.argmax(1)==inputs['labels']).sum().item()

  return loss / len(eval_dataset), acc / len(eval_dataset)

if __name__ == '__main__':
  #root_path = "/content/drive/MyDrive/Colab Notebooks/buddy/data"
  root_path = "/content/drive/MyDrive/Colab Notebooks/buddy"
  data_path = f"{root_path}/data2/wellness_dialog_for_text_classification_test.txt"
  checkpoint_path = f"{root_path}/checkpoint"
  save_ckpt_path = f"{checkpoint_path}/kobert-wellness-text-classification-dataX-191.pth"
  #model_name_or_path = "monologg/koelectra-base-discriminator"

  n_epoch = 50  # Num of Epoch
  batch_size = 16  # 배치 사이즈
  ctx = "cuda" if torch.cuda.is_available() else "cpu"
  device = torch.device(ctx)
  model_names=["kobert"]
  for model_name in model_names:
    eval_loss, eval_acc = evaluate(model_name, device, batch_size, data_path)
    print(f'\tLoss: {eval_loss:.4f}(valid)\t|\tAcc: {eval_acc * 100:.1f}%(valid)')

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

load pretrain from: /content/drive/MyDrive/Colab Notebooks/buddy/checkpoint/kobert-wellness-text-classification-dataX-191.pth, epoch=51


Evaluating: 100%|██████████| 26/26 [00:04<00:00,  5.90it/s]

	Loss: 0.2167(valid)	|	Acc: 46.6%(valid)
